In [1]:
# !pip install torch torchvision
# !pip install wandb
# !pip install nbformat

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [6]:
# hyperparameters

batch_size = 64
learning_rate = 0.001
epochs = 30

In [7]:
import wandb
# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="zeri-university-of-michigan",
    # Set the wandb project where this run will be logged.
    project="mnist-basic",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "architecture": "CNN",
        "dataset": "MNIST",
        "epochs": epochs,
    },
    # run name
    name="mnist-basic",
)




wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: zeri (zeri-university-of-michigan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


In [9]:

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [11]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        return self.fc(x)


In [12]:

# 4. Initialize model, loss, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [13]:
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import numpy as np

for epoch in range(epochs):
    # -------- TRAINING --------
    model.train()
    running_loss = 0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device)

        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    # -------- EVALUATION --------
    model.eval()
    correct = 0
    total = 0
    eval_loss = 0
    all_probs = []
    all_targets = []

    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)

            loss = criterion(outputs, targets)
            eval_loss += loss.item()

            probs = torch.softmax(outputs, dim=1)  # For multi-class
            all_probs.append(probs.cpu())
            all_targets.append(targets.cpu())

            _, predicted = outputs.max(1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

    # Convert to NumPy for PR-AUC
    all_probs = torch.cat(all_probs).numpy()          # shape: [N, 10]
    all_targets = torch.cat(all_targets).numpy()      # shape: [N]
    all_targets_bin = label_binarize(all_targets, classes=np.arange(10))  # [N, 10]

    pr_auc = average_precision_score(all_targets_bin, all_probs, average='macro')
    accuracy = 100 * correct / total
    avg_eval_loss = eval_loss / len(test_loader)

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {avg_train_loss:.4f} "
          f"Val Loss: {avg_eval_loss:.4f} "
          f"Accuracy: {accuracy:.2f}% "
          f"PR-AUC: {pr_auc:.4f}")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "val_loss": avg_eval_loss,
        "val_accuracy": accuracy,
        "val_pr_auc": pr_auc
    })

Epoch [1/30] Train Loss: 0.3480 Val Loss: 0.1844 Accuracy: 94.51% PR-AUC: 0.9850
Epoch [2/30] Train Loss: 0.1558 Val Loss: 0.1295 Accuracy: 96.07% PR-AUC: 0.9924
Epoch [3/30] Train Loss: 0.1071 Val Loss: 0.1028 Accuracy: 96.83% PR-AUC: 0.9947
Epoch [4/30] Train Loss: 0.0814 Val Loss: 0.0878 Accuracy: 97.33% PR-AUC: 0.9960
Epoch [5/30] Train Loss: 0.0641 Val Loss: 0.0873 Accuracy: 97.38% PR-AUC: 0.9962
Epoch [6/30] Train Loss: 0.0527 Val Loss: 0.0801 Accuracy: 97.55% PR-AUC: 0.9966
Epoch [7/30] Train Loss: 0.0435 Val Loss: 0.0706 Accuracy: 97.79% PR-AUC: 0.9972
Epoch [8/30] Train Loss: 0.0345 Val Loss: 0.0730 Accuracy: 97.80% PR-AUC: 0.9969
Epoch [9/30] Train Loss: 0.0295 Val Loss: 0.0722 Accuracy: 97.97% PR-AUC: 0.9969
Epoch [10/30] Train Loss: 0.0239 Val Loss: 0.0717 Accuracy: 97.83% PR-AUC: 0.9973
Epoch [11/30] Train Loss: 0.0197 Val Loss: 0.0729 Accuracy: 97.84% PR-AUC: 0.9973
Epoch [12/30] Train Loss: 0.0165 Val Loss: 0.0771 Accuracy: 97.77% PR-AUC: 0.9972
Epoch [13/30] Train Loss:

In [14]:

# 6. Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, targets in test_loader:
        data, targets = data.to(device), targets.to(device)
        outputs = model(data)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 97.91%


In [15]:
artifact = wandb.Artifact(
    name="mnist-basic",
    type="model",
    metadata={
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "architecture": "CNN",
        "dataset": "MNIST",
        "epochs": epochs,
    }
)

In [16]:
# save model
torch.save(model.state_dict(), "mnist-basic.pth")
artifact.add_file("mnist-basic.pth")
run.log_artifact(artifact)


<Artifact mnist-basic>

In [17]:
run.finish()

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▆▇▇▇████████▇███████████████
val_loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▂▂▂▂▂▃▂▂▂▂▃▃▃▃▃▃
val_pr_auc,▁▅▇▇▇█████████████████████████
epoch,30
train_loss,0.00325
val_accuracy,97.91
val_loss,0.10316
val_pr_auc,0.9972
